<a href="https://colab.research.google.com/github/FelipeMoreira01/Machine-Learning-FCC/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     |████████████████████████████████| 399.9MB 39kB/s 
     |████████████████████████████████| 4.0MB 48.5MB/s 
     |████████████████████████████████| 471kB 60.2MB/s 
     |████████████████████████████████| 4.0MB 41.9MB/s 
     |████████████████████████████████| 12.2MB 261kB/s 
ERROR: tensorflow 2.4.0 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
2.5.0-dev20210108


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-01-08 12:51:13--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv          [   <=>              ] 349.84K   347KB/s    in 1.0s    

2021-01-08 12:51:15 (347 KB/s) - ‘train-data.tsv’ saved [358233]

--2021-01-08 12:51:15--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv          [  <=>               ] 115.

In [3]:
COLS = ['type', 'message']

train_data = pd.read_csv(train_file_path, 
                       sep='\t', 
                       header=None,
                       names=COLS)
test_data = pd.read_csv(test_file_path,
                      sep='\t',
                      header=None,
                      names=COLS)
train_labels = train_data.pop('type')
test_labels = test_data.pop('type')

In [4]:
train_data.head()

,message
0,ahhhh...just woken up!had a bad dream about u ...
1,you can never do nothing
2,"now u sound like manky scouse boy steve,like! ..."
3,mum say we wan to go then go... then she can s...
4,never y lei... i v lazy... got wat? dat day ü ...


In [5]:
train_labels = train_labels.astype('category').cat.codes #{0: ham, 1: spam}
test_labels = test_labels.astype('category').cat.codes

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data.values, train_labels.values))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data.values, test_labels.values))
type(train_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [7]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  [b'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.']
label:  0


In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [[b'ugh my leg hurts. musta overdid it on mon.']
 [b'the hair cream has not been shipped.']
 [b"not really dude, have no friends i'm afraid :("]]

labels:  [0 0 0]


Words embedding: transform text to int

In [10]:
VOCAB_SIZE = 1000

encoder = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE
)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [11]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in', 'is',
       'me', 'my', 'for', 'your', 'of', 'it', 'call', 'have', 'on'],
      dtype='<U15')

In [12]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  1,  12,   1,   1,   1,   1,  16,  19,   1,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  6, 592,   1, 108,  25,  97,   1,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [ 25, 144, 534,  18,  41, 226,  23,   1,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0]])

Create Model

In [30]:
model = keras.Sequential([
                          encoder,
                          keras.layers.Embedding(
                              input_dim=len(encoder.get_vocabulary()),
                              output_dim=64,
                              mask_zero=True),
                          keras.layers.Bidirectional(keras.layers.LSTM(64)),
                          keras.layers.Dense(64, activation='relu'),
                          keras.layers.Dense(1, activation='sigmoid')
])

In [31]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [33]:
#training

history = model.fit(train_dataset, epochs=10,
                   validation_data=test_dataset,
                   validation_steps=30)

Epoch 1/10
66/66 [==============================] - 18s 152ms/step - loss: 0.6757 - accuracy: 0.8520 - val_loss: 0.5902 - val_accuracy: 0.8657
Epoch 2/10
66/66 [==============================] - 7s 110ms/step - loss: 0.5470 - accuracy: 0.8581
Epoch 3/10
66/66 [==============================] - 7s 109ms/step - loss: 0.4123 - accuracy: 0.8692
Epoch 4/10
66/66 [==============================] - 7s 105ms/step - loss: 0.2919 - accuracy: 0.8681
Epoch 5/10
66/66 [==============================] - 7s 107ms/step - loss: 0.1666 - accuracy: 0.9613
Epoch 6/10
66/66 [==============================] - 7s 110ms/step - loss: 0.1063 - accuracy: 0.9778
Epoch 7/10
66/66 [==============================] - 7s 110ms/step - loss: 0.0791 - accuracy: 0.9812
Epoch 8/10
66/66 [==============================] - 7s 108ms/step - loss: 0.0541 - accuracy: 0.9885
Epoch 9/10
66/66 [==============================] - 7s 107ms/step - loss: 0.0508 - accuracy: 0.9864
Epoch 10/10
66/66 [==============================] - 7s 1

In [34]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

22/22 [==============================] - 1s 28ms/step - loss: 0.0608 - accuracy: 0.9849
Test Loss: 0.06083543226122856
Test Accuracy: 0.9849137663841248


In [35]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    prediction = []

    pred = model.predict(np.array([pred_text]))
    prediction.append(pred.flatten()[0])
    if pred[0] > 0.5:
        prediction.append('spam')
    else:
        prediction.append("ham")



    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.0064409673, 'ham']


In [37]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction[1])
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


ham
spam
ham
spam
spam
ham
ham
You passed the challenge. Great job!
